In [40]:
import pandas as pd
from astropy.time import Time
from itertools import cycle

In [41]:
# Corrected file path with .csv extension
file = r"/home/arya/Downloads/B_team_coords_MJDs.csv"

# Read the CSV file
df = pd.read_csv(file)


In [42]:
df.head()

,Index,RA,DEC,TOA
0,0,RA: 190.82262499999987,DEC: -63.905138888888885,MJD TOA: 59105.19956770833
1,1,RA: 190.8198333333333,DEC: -64.87469444444444,MJD TOA: 59105.199738564814
2,2,RA: 189.85924999999997,DEC: -64.14383333333335,MJD TOA: 59105.19886953704
3,3,RA: 191.77629166666665,DEC: -63.654444444444444,MJD TOA: 59105.1968799537
4,4,RA: 237.60416666666666,DEC: -46.17219444444445,MJD TOA: 59105.22789008102


In [43]:
# Function to generate a sequence of alphabet tags (A-Z, AA-ZZ, etc.)
def generate_alphabet_sequence():
    alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    for size in range(1, 4):  # Extend to 3-letter combinations if needed
        for combo in product(alphabet, repeat=size):
            yield "".join(combo)

In [44]:
# Clean the RA and DEC columns by removing the text prefix
# and converting the values to float with 8 decimal places
df['RA'] = df['RA'].str.extract(r'([-+]?\d*\.\d+)').astype(float).round(8)
df['DEC'] = df['DEC'].str.extract(r'([-+]?\d*\.\d+)').astype(float).round(8)

# Extract the MJD values from the `TOA` column
df['MJD'] = df['TOA'].str.extract(r'(\d+\.\d+)').astype(float)

# Convert MJD to UTC using astropy
times = Time(df['MJD'], format='mjd')
df['UTC TOA'] = times.iso  # Convert to ISO format for readability

# Extract date part from UTC TOA and format it as YYYYMMDD
df['DATE'] = df['UTC TOA'].str[:10].str.replace('-', '')  # Remove hyphens

# Create the 'Name' column in the format ARYAYYYYMMDD
df['Name'] = df.groupby('DATE', group_keys=False).apply(assign_alphabet_sequence)


# Drop the intermediate DATE column
df = df.drop(columns=['TOA', 'DATE'])

# Rearrange the column order
df = df[['Name', 'RA', 'DEC', 'MJD', 'UTC TOA']]


/tmp/ipykernel_757504/114945874.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['Name'] = df.groupby('DATE', group_keys=False).apply(assign_alphabet_sequence)


In [48]:
# Display the first few rows to verify
df.head

<bound method NDFrame.head of     Index         RA       DEC           MJD                  UTC TOA  \
0       0  190.82262 -63.90514  59105.199568  2020-09-13 04:47:22.650   
1       1  190.81983 -64.87469  59105.199739  2020-09-13 04:47:37.412   
2       2  189.85925 -64.14383  59105.198870  2020-09-13 04:46:22.328   
3       3  191.77629 -63.65444  59105.196880  2020-09-13 04:43:30.428   
4       4  237.60417 -46.17219  59105.227890  2020-09-13 05:28:09.703   
..    ...        ...       ...           ...                      ...   
69     69  326.98571 -80.10400  59110.542022  2020-09-18 13:00:30.683   
70     70  334.45992 -80.32714  59110.543747  2020-09-18 13:02:59.757   
71     71  333.59571  -8.14019  59111.476835  2020-09-19 11:26:38.577   
72     72  329.39829 -80.84278  59111.587025  2020-09-19 14:05:18.995   
73     73  326.84112 -80.58811  59111.588913  2020-09-19 14:08:02.106   

             Name  
0   ARYA20200913A  
1   ARYA20200913B  
2   ARYA20200913C  
3   ARYA20200

In [49]:
output_file = r"/home/arya/Downloads/B_team_coords_MJDs_with_UTC.csv"
df.to_csv(output_file, index=False)
